In [7]:
import os
from scipy import spatial
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

# Loading data

In [2]:
def get_articles_paths(input_path):
    paths = []
    if input_path[-1] != '/':
        input_path += '/'
    
    items = os.listdir(input_path)
    for item in items:
        extension = item[-3:]
        if extension in ['txt']:
            paths.append(input_path + item)
        else:
            # we assume that item is a directory
            paths.extend(get_articles_paths(input_path + item))
    return paths

In [3]:
path = '/content/drive/MyDrive/Wikipedia'
paths = get_articles_paths(path)
print(len(paths))

41063


In [5]:
data = {'id':[], 'url':[], 'title':[], 'text':[]}
for i,path in enumerate(paths):
  print(i, 'from', len(paths))
  with open(path, 'r') as f:
    article = f.read()
    soup = BeautifulSoup(article, 'lxml')
    id = soup.find('doc')['id']
    url = soup.find('doc')['url']
    title = soup.find('doc')['title']
    text = soup.text
    data['id'].append(id)
    data['url'].append(url)
    data['title'].append(title)
    data['text'].append(text)

0 from 41063
1 from 41063
2 from 41063
3 from 41063
4 from 41063
5 from 41063
6 from 41063
7 from 41063
8 from 41063
9 from 41063
10 from 41063
11 from 41063
12 from 41063
13 from 41063
14 from 41063
15 from 41063
16 from 41063
17 from 41063
18 from 41063
19 from 41063
20 from 41063
21 from 41063
22 from 41063
23 from 41063
24 from 41063
25 from 41063
26 from 41063
27 from 41063
28 from 41063
29 from 41063
30 from 41063
31 from 41063
32 from 41063
33 from 41063
34 from 41063
35 from 41063
36 from 41063
37 from 41063
38 from 41063
39 from 41063
40 from 41063
41 from 41063
42 from 41063
43 from 41063
44 from 41063
45 from 41063
46 from 41063
47 from 41063
48 from 41063
49 from 41063
50 from 41063
51 from 41063
52 from 41063
53 from 41063
54 from 41063
55 from 41063
56 from 41063
57 from 41063
58 from 41063
59 from 41063
60 from 41063
61 from 41063
62 from 41063
63 from 41063
64 from 41063
65 from 41063
66 from 41063
67 from 41063
68 from 41063
69 from 41063
70 from 41063
71 from 41063
72

KeyboardInterrupt: ignored

In [8]:
df = pd.DataFrame.from_dict(data, orient='columns')
df

,id,url,title,text
0,28,http://pl.wikipedia.org/wiki/?curid=28,Abstract Syntax Notation One,\nAbstract Syntax Notation One\n\nASN.1 (skrót...
1,16,http://pl.wikipedia.org/wiki/?curid=16,Association for Computing Machinery,\nAssociation for Computing Machinery\n\nAssoc...
2,8,http://pl.wikipedia.org/wiki/?curid=8,Aksjomat,"\nAksjomat\n\nAksjomat (postulat, pewnik) (gr...."
3,29,http://pl.wikipedia.org/wiki/?curid=29,Algorytm,\nAlgorytm\n\nAlgorytm – w matematyce oraz inf...
4,22,http://pl.wikipedia.org/wiki/?curid=22,Alkiny,"\nAlkiny\n\nAlkiny (zwyczajowo ""acetyleny"") – ..."
...,...,...,...,...
468,699,http://pl.wikipedia.org/wiki/?curid=699,Barykada,"\nBarykada\n\nBarykada (fr. ""barricade"") – dor..."
469,695,http://pl.wikipedia.org/wiki/?curid=695,Baśń,"\nBaśń\n\nBaśń (często mylona z bajką, krótkim..."
470,692,http://pl.wikipedia.org/wiki/?curid=692,Bohdan Chmielnicki,\nBohdan Chmielnicki\n\nBohdan Zenobi Chmielni...
471,704,http://pl.wikipedia.org/wiki/?curid=704,Baccio Bandinelli,\nBaccio Bandinelli\n\nBaccio Bandinelli właśc...


In [14]:
def divide_into_par(text, threshold=10):
  splits = text.split('\n')
  pars = []
  par = ''
  for split in splits:
    words = split.split(' ')
    if len(words) > 0 and len(words) < threshold:
      if len(par) > 0:
        pars.append(par)
        par = ''
    elif len(words) > threshold:
      par += split
  if len(pars) > 0:
    return pars
  return text

In [15]:
df['pars'] = df['text'].apply(divide_into_par)

In [16]:
df.head()

,id,url,title,text,pars
0,28,http://pl.wikipedia.org/wiki/?curid=28,Abstract Syntax Notation One,\nAbstract Syntax Notation One\n\nASN.1 (skrót...,"[ASN.1 (skrót od ""Abstract Syntax Notation One..."
1,16,http://pl.wikipedia.org/wiki/?curid=16,Association for Computing Machinery,\nAssociation for Computing Machinery\n\nAssoc...,"[Association for Computing Machinery (""ACM"") t..."
2,8,http://pl.wikipedia.org/wiki/?curid=8,Aksjomat,"\nAksjomat\n\nAksjomat (postulat, pewnik) (gr....","[Aksjomat (postulat, pewnik) (gr. αξιωμα [""aks..."
3,29,http://pl.wikipedia.org/wiki/?curid=29,Algorytm,\nAlgorytm\n\nAlgorytm – w matematyce oraz inf...,[Algorytm – w matematyce oraz informatyce skoń...
4,22,http://pl.wikipedia.org/wiki/?curid=22,Alkiny,"\nAlkiny\n\nAlkiny (zwyczajowo ""acetyleny"") – ...","[Alkiny (zwyczajowo ""acetyleny"") – grupa organ..."


In [17]:
df = df.explode('pars')
df.head()

,id,url,title,text,pars
0,28,http://pl.wikipedia.org/wiki/?curid=28,Abstract Syntax Notation One,\nAbstract Syntax Notation One\n\nASN.1 (skrót...,"ASN.1 (skrót od ""Abstract Syntax Notation One""..."
1,16,http://pl.wikipedia.org/wiki/?curid=16,Association for Computing Machinery,\nAssociation for Computing Machinery\n\nAssoc...,"Association for Computing Machinery (""ACM"") to..."
1,16,http://pl.wikipedia.org/wiki/?curid=16,Association for Computing Machinery,\nAssociation for Computing Machinery\n\nAssoc...,ACM zostało utworzone 15.IX.1947 roku w mieści...
1,16,http://pl.wikipedia.org/wiki/?curid=16,Association for Computing Machinery,\nAssociation for Computing Machinery\n\nAssoc...,Członkowie w ACM dzielą się na studenckich i p...
1,16,http://pl.wikipedia.org/wiki/?curid=16,Association for Computing Machinery,\nAssociation for Computing Machinery\n\nAssoc...,ACM jest kierowane przez radę 16 członków. W s...


In [18]:
df = df.reset_index(drop=True)

# Searching with SentenceTransformers

In [19]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 1.5MB 24.5MB/s 
     |████████████████████████████████| 1.1MB 57.7MB/s 
     |████████████████████████████████| 890kB 57.2MB/s 
     |████████████████████████████████| 2.9MB 55.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.0-cp36-none-any.whl size=102655 sha256=0b8e811208e155007dfadfd4be21ce98f053b2ec8358b1e15eee0a53ed6cf6e2
  Stored in directory: /root/.cache/pip/wheels/ff/76/65/50258d8b7930e909ea2f5bd006a23d520a16765af13ab45bb3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a2e916873f9d472cdeca5707e880203d37038a8e1bfdcce7fc646ed6c0779de5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [20]:
model = SentenceTransformer('distiluse-base-multilingual-cased')

100%|██████████| 504M/504M [00:29<00:00, 17.2MB/s]


In [21]:
sentences = df['pars']
sentence_embeddings = model.encode(sentences)

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


In [22]:
def prep_sbert(model, query):
  query_emb = model.encode(query)
  return query_emb

In [26]:
def search(model, preprocess_func, query, n):
  query_emb = preprocess_func(model, query)
  results = []
  for i, emb in enumerate(sentence_embeddings):
    result = 1 - spatial.distance.cosine(emb, query_emb)
    results.append(result)
  results = np.array(results)
  indices = results.argsort()[::-1]
  best_articles = df.iloc[indices[:n]]
  for index, article in best_articles.iterrows():
    print('\nTitle:', article['title'])
    print('Paragraph:', article['pars'])
    print('Url:', article['url'])
    print('Index:', index)
  print(results[indices[:n]])

In [24]:
df['title'].unique()[100:200]

array(['Akapit', 'Aurelian (imię)', 'Adolf Hitler', 'Atopia', 'Ateizm',
       'Auto-Ordnance', 'Aleksander Sołżenicyn', 'Adrenalina',
       'Aleksander Gieysztor', 'Rijad', 'Absynt', 'Argentyna',
       'Anschluss Austrii', 'Alfabet husycki', 'Andreas Vesalius',
       'Ada (imię)', 'Alfred Hitchcock', 'Athene (system operacyjny)',
       'Armia Krajowa', 'Asembler x86', 'Agnostycyzm', 'Asymilacja',
       'Australia', 'Advanced Research Projects Agency Network',
       'Antropometria', 'Akwarela', 'André Gide', 'Active Server Pages',
       'Arabia Saudyjska', 'Alan Cox', 'Bolesław Chrobry (powieść)',
       'Basic English', 'Bantustan', 'Biskupin (powiat żniński)',
       'Beskid Makowski', 'Bieszczady', 'Brigitte Bardot',
       'Beskidy Zachodnie', 'Bernardo Bellotto', 'Bezprym', 'Bobrzanie',
       'Brom', 'Boudika', 'Beskidy', 'Barwy pastelowe', 'Babia Góra',
       'Bozon', 'Budżet', 'Mjanma', 'Brytowie', 'Błogosławiony',
       'Bedřich Hrozný', 'Broszura', 'Burowie', 'Blende

In [27]:
query = 'Sposób malowania obrazów farbami wodnymi'
search(model, prep_sbert, query, 20)


Title: Akwarela
Paragraph: Akwarela − nazwa techniki malarskiej i jednocześnie nazwa obrazów wykonywanych tą techniką. Technika ta polega na malowaniu rozcieńczonymi w wodzie pigmentami na porowatym papierze, który szybko wchłania wodę. Rozcieńczone pigmenty nie pokrywają całkowicie faktury papieru, lecz pozostawiają ją dobrze widoczną. Z tego względu na efekt końcowy bardzo duży wpływ ma barwa i faktura samego papieru.Obrazy malowane tą techniką mają zwykle delikatną, zwiewną kolorystykę, gdyż możliwe tutaj do uzyskania barwy to głównie barwy pastelowe. Technikę tę stosuje się do tematów, które wymagają tego rodzaju kolorystyki i jednocześnie nie wymagają "cyzelowania" detali - stosuje się ją więc do pejzaży, "zwiewnych" portretów itp.Akwarela jest jedną z tańszych technik malarskich, lecz jest ona bardzo trudna, gdyż nie daje prawie żadnej możliwości dokonywania poprawek i retuszy. W szczególności, nie można usunąć namalowanego fragmentu, a jedynie przerobić go, i to też w niewielki